In [6]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import requests
import json
from config import gkey

In [7]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://www.colorado.com/colorado-breweries"


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [8]:
names_list = []
address_list = []
website_url_list = []
browser.visit(url)
time.sleep(1)
browser.find_by_tag("span[class*='icon icon--cross']").click()
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
address_soup = soup.find_all("span", class_ = ("grid--result--address ng-binding"))
names_soup = soup.find_all("h3", class_ = ("grid--result--name ng-binding ng-scope"))
website_soup = soup.find_all("p", class_ = ("grid--result-cta"))
buttons = browser.find_by_tag("button[class*= 'pagination-call pagination-call-next']")
buttons.click()

In [9]:
list_length = range(0, round(300/10))
for y in list_length:
    try: 
        if browser.find_by_tag("button[class*= 'pagination-call pagination-call-next']") == True:
            buttons.click()
        else:
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            address_soup = soup.find_all("span", class_ = ("grid--result--address ng-binding"))
            names_soup = soup.find_all("h3", class_ = ("grid--result--name ng-binding ng-scope"))
            website_soup = soup.find_all("p", class_ = ("grid--result-cta"))
            for x in names_soup:
                names_list.append(x.text)
            for x in address_soup:
                address_list.append(x.text)
            for x in website_soup:
                for link in x:
                    if link.has_attr('href'):
                        website_url_list.append(link.attrs['href'])
                    else:
                        website_url_list.append("NaN")
            buttons = browser.find_by_tag("button[class*= 'pagination-call pagination-call-next']")
            buttons.click()
    
    except:
        print("error")
display(address_list)
display(names_list)

error
error
error
error


['133 Remington Street, Fort Collins, CO 80524',
 '104 Village Pl., Dillon, CO 80435',
 '2161 Blake Street, Denver, CO 80205',
 '800 E. Lincoln Ave, Fort Collins, CO 80524',
 '41290 B-2 & B-3 US HWY 6, Vail, CO 81657',
 '1480 Cascade Ave., Loveland, CO 80538',
 '577 Twenty-Second Street, Georgetown, CO 80444',
 '8665 N. Sheridan Blvd., Westminster, CO 80003',
 '4910 Nautilus Ct., Boulder, CO 80301',
 '1132 W Dillon Road, Louisville, CO 80027',
 '1150 Pine Street, Unit B, Louisville, CO 80027',
 '471 Kalamath St., Denver, CO 80204',
 '3121 A North El Paso Street, Colorado Springs, CO 80907',
 '170 S. Main Street, Brighton, CO 80601',
 '1025 Delaware Avenue, Longmont, CO 80501',
 '2070 County Road 12 South, Alamosa, CO 81101',
 '2801 Tower Road, Aurora, CO 80011',
 '4460 West 29th Avenue, Denver, CO 80212',
 '531 Silverthorne Lane, Silverthorne, CO 80498',
 '393 N Washington Ave, Golden, CO 80403',
 '6575 S Greenwood Plaza Blvd., Englewood, CO 80111',
 '846 East Main, Montrose, CO 81041'

['Equinox Brewing',
 "Pug Ryan's Steak House Brewery",
 'Blue Moon Brewing Co. at the Sandlot',
 'Odell Brewing Company',
 'Vail Brewing Company',
 '5030Local',
 'Cabin Creek Brewing',
 'Yak & Yeti - Westminster',
 'Avery Brewing Company',
 'Mother Tucker Brewery',
 'Gravity Brewing',
 'Crazy Mountain Brewery',
 'Storybook Brewing',
 'Floodstage Ale Works',
 'Grossen Bart Brewery',
 'The Colorado Farm Brewery',
 'Dry Dock-North Dock',
 'Hogshead Brewery',
 "Bakers' Brewery",
 'Cannonball Creek Brewing Co.',
 'CB & Potts',
 'Horsefly Brewing Company',
 'Brewability',
 'Crow Hop Brewing',
 'Whistle Pig Brewing Company',
 'Great Divide Brewing Company',
 'Echo Brewing',
 'Bull and Bush Brewery',
 'Paradox Beer Company',
 "Coopersmith's Pub and Brewing and Billiards",
 'HighSide Brewing',
 'Colorado Boy Pub & Brewery',
 'Butcher Knife Brewing',
 'Lumpy Ridge Brewing Co.',
 'Rain Brews',
 'WeldWerks Brewing Company',
 'Mobb Mountain Distillers',
 'Baere Brewing Co.',
 'Southern Sun Pub & Br

In [18]:
display(len(address_list))
display(len(names_list))
display(len(website_url_list))

262

262

262

In [19]:
addresses = []
for x in address_list:
    addresses.append(x.replace(" ", "+"))

In [20]:
lats = []
lngs = []
g_url = "https://maps.googleapis.com/maps/api/geocode/json?address="
for x in addresses:
    try:
        response = requests.get(f'{g_url}{x}&key={gkey}')
        places_data = response.json()
        lats.append(places_data["results"][0]['geometry']['location']['lat'])
        lngs.append(places_data["results"][0]['geometry']['location']['lng'])
    except:
        lats.append("NaN")
        lngs.append("NaN")

In [21]:
breweries_df = pd.DataFrame({"Brewery" : names_list, "Address" : address_list,"Website" : website_url_list})
breweries_df["Latitudes"] = lats
breweries_df["Longitudes"] = lngs
breweries_df = breweries_df.set_index("Brewery")
breweries_df = breweries_df.dropna()
breweries_df = breweries_df.drop_duplicates()
cleaned_breweries = breweries_df[breweries_df["Longitudes"] != "NaN"]
cleaned_breweries.to_csv("../Resources/cleaned_breweries.csv")
cleaned_breweries


,Address,Website,Latitudes,Longitudes
Brewery,,,,
Equinox Brewing,"133 Remington Street, Fort Collins, CO 80524",http://www.equinoxbrewing.com,40.5863,-105.076
Pug Ryan's Steak House Brewery,"104 Village Pl., Dillon, CO 80435",https://www.pugryans.com,39.628,-106.046
Blue Moon Brewing Co. at the Sandlot,"2161 Blake Street, Denver, CO 80205",http://millercoorsblog.com,39.7557,-104.993
Odell Brewing Company,"800 E. Lincoln Ave, Fort Collins, CO 80524",http://www.odellbrewing.com/,40.5895,-105.063
Vail Brewing Company,"41290 B-2 & B-3 US HWY 6, Vail, CO 81657",http://vailbrewingco.com,39.6206,-106.47
...,...,...,...,...
300 Suns Brewing,"335 1st Ave., Longmont, CO 80501",http://300sunsbrewing.com/,40.1599,-105.101
Finkel & Garf Brewing Co.,"5455 Spine Road, Boulder, CO 80301",http://finkelandgarf.com,40.074,-105.203
Pumphouse Brewery,"540 Main Street, Longmont, CO 80501",http://www.pumphousebrewery.com,40.1687,-105.102
